# LawChat

**Use natural language to query a library of previosuly cited legal cases**

We make use of LangChain for:

- managing the Vector Store: Cassandra
- managing the embeddings: OpenAIEmbeddings
- loading data from web pages
- FLARE chain

## Configuration

**Create your `.env1` file**

1. Copy the .env.example file to `.env`
2. Specify your Astra and openAI parameters.

In [1]:
import os
from dotenv import load_dotenv

class Settings():
    load_dotenv()
    ASTRA_DB_KEYSPACE = os.environ['ASTRA_DB_KEYSPACE']
    ASTRA_DB_SECURE_BUNDLE_PATH = os.environ['ASTRA_DB_SECURE_BUNDLE_PATH']
    ASTRA_DB_APPLICATION_TOKEN = os.environ['ASTRA_DB_APPLICATION_TOKEN']
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Astra DB Connectivity

In [3]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.cqlengine import connection


# load settings and keys
settings = Settings


def getCluster():
    """
    Create a Cluster instance to connect to Astra DB.
    Uses the secure-connect-bundle and the connection secrets.
    """
    cloud_config = {"secure_connect_bundle": settings.ASTRA_DB_SECURE_BUNDLE_PATH}
    auth_provider = PlainTextAuthProvider("token", settings.ASTRA_DB_APPLICATION_TOKEN)
    return Cluster(cloud=cloud_config, auth_provider=auth_provider)


def get_astra():
    """
    This function is used by LangChain Vectorstore.
    """
    cluster = getCluster()
    astraSession = cluster.connect()
    return astraSession, settings.ASTRA_DB_KEYSPACE


def initSession():
    """
    Create the DB session and return it to the caller.
    Most important, the session is also set as default and made available
    to the object mapper through global settings. I.e., no need to actually
    do anything with the return value of this function.
    """
    cluster = getCluster()
    session = cluster.connect()
    session.set_keyspace("lawchat")
    connection.register_connection("my-astra-session", session=session)
    connection.set_default_connection("my-astra-session")
    return connection

## Load library data

We are using data from the Australasian Legal Information Institute. AustLII maintains collections of primary materials: legislation and court judgments ("case law"). 

For this project, we are sourcing case law from the Supreme Court of New South Wales.

#### Utility functions

In [4]:
import re

"""
Function to clean text from web pages
"""
def clean_text(text: str):
    # Normalize line breaks to \n\n (two new lines)
    text = text.replace("\r\n", "\n\n")
    text = text.replace("\r", "\n\n")

    # Replace two or more spaces with a single space
    text = re.sub(" {2,}", " ", text)

    # Remove leading spaces before removing trailing spaces
    text = re.sub("^[ \t]+", "", text, flags=re.MULTILINE)

    # Remove trailing spaces before removing empty lines
    text = re.sub("[ \t]+$", "", text, flags=re.MULTILINE)

    # Remove empty lines
    text = re.sub("^\s+", "", text, flags=re.MULTILINE)

    return text


In [5]:
import tiktoken

"""
Function to calculate the number of tokens in a text string.
"""

encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_string(string: str) -> int:
    num_tokens = len(encoding.encode(string))
    return num_tokens

#### Define the Vector Store

In [6]:
from langchain.vectorstores import Cassandra
from langchain.embeddings.openai import OpenAIEmbeddings

# define Embedding model
embeddings = OpenAIEmbeddings()

# Set up the vector store
print("Setup Vector Store")
session, keyspace = get_astra()
vectorstore = Cassandra(
    embedding=embeddings,
    session=session,
    keyspace=keyspace,
    table_name="nswsc",
)

Setup Vector Store


#### Get data files

We load a number of HTML pages using the LangChain WebBaseLoader. Each of those pages contains lots of superfluous content so we extract only the rfelvant article.

In [7]:
from langchain.document_loaders import WebBaseLoader

urls = [
    "https://www.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/1998/423.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/2002/949.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/1998/4.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/2005/1181.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/1998/483.html"
    ]
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
}

print("Loading Data")
url_loaders = WebBaseLoader(urls, header_template=headers)
data = url_loaders.load()

"""
Extract only the actual Article content from the web page and clean
"""
print("Cleaning Data")
for i, d in enumerate(data):
    d.page_content = ""
    source = d.metadata['source']
    thedoc = WebBaseLoader(source, header_template=headers).scrape()
    # extract only the Article content from the web page
    td = thedoc.find("article", {"class": "the-document"}).text
    d.page_content = clean_text(td)
    data[i] = d

print (f"Number of documents: {len(urls)}")
print (f"Number of tokens: {num_tokens_from_string(data[0].page_content)}")


Loading Data
Cleaning Data
Number of documents: 5
Number of tokens: 33772


#### Split the data into chunks

In [8]:
from langchain.text_splitter import TokenTextSplitter

CHUNK_SIZE = 500

# Chunk the data
print("Splitting Data")
text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=50)
docs = text_splitter.split_documents(data)
print(f"Number of chunks: {len(docs)}")

Splitting Data
Number of chunks: 226


#### Store data and embeddings in Astra DB

In [9]:
print("Adding texts to Vector Store")
texts, metadatas = zip(*((doc.page_content, doc.metadata) for doc in docs))
vectorstore.add_texts(texts=texts, metadatas=metadatas)

Adding texts to Vector Store


['de7f129c91ef47a18b08cab3d6faa700',
 'ea2ec44d53304eceb95045a13eae09b3',
 'f64a473ac3bd4fc080b09d9bc8c3b94e',
 '83372b245a674180bc51d6a90a9a9eef',
 '18594bcb70244ecdb7f8aa228e5807f9',
 '2b54cca552c747cd9bd441292bc04929',
 'e3f256784609490ab01af4c76f2db541',
 'c1b79a31730948d8810821b0808b2a43',
 '47b55fbfd17a410a9d5c6a070ff9727d',
 'e9fb04433e9b460ab5ec3bc045640ea6',
 'bbe5dc0fb9e44bf0845de20c6109e1fa',
 '07e32b4f68ff4985958224d23beb6ad2',
 'e73d56de3953448b8fec97a3d2518b52',
 '98f353d9878e431186f0f41bd5c221a8',
 '70dfdb7c24f04f13b3ccebb46c644361',
 'f3cc9eb73db14fb090fe96cd431c58d4',
 '58c0d833871547069757ba09a24ed1ab',
 '5a92bd9685264e89ab619aab0356c61e',
 '9e9e3e0e513447e6ab1e7148ea06dae8',
 '4862ca1901184506b362922331182f8c',
 'fd8286ba038745788a9cd23a9ea9b82f',
 'a052dded93bb42c39a5a7e590b5da84b',
 '1d716cd63fe049faa0183a2a758a7e1e',
 'c3f46691b06d49a18e32fa4073320a7d',
 '5847939d6dd94a5eaefc4374d875ef62',
 'd581a44ac8e8431d8f68650688c13ba7',
 '207730b6c0bb49fea323f4f9509249e6',
 

## Query

#### Define the Retriever
by setting "k" in kwargs, we can control how many results are retrieved from the vector store and passed into the Flare chain.

- Ideally, we should not provide a "K", but the only way I can get this to work is limit the K above to a single result in order to not breach the token limit of 4096

In [10]:
import sys
from langchain.chains import FlareChain
from langchain.chat_models import ChatOpenAI

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
#retriever = vectorstore.as_retriever()

#### Define FLARE chain

Appears that the model attribute is ignored as an error is thrown if more than 4096 tokens are supplied.

- the only way I can get this to work is to either:

    - limit the K above to a single result in order to not breach the token limit. But that means we are not retreiving enough context from the data.
    - reduce the chunk size to create smaller chunks and thus reduce the number of tokens passed to the LLM.

In [11]:
flare = FlareChain.from_llm(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    retriever=retriever,
    max_generation_len=164,
    min_prob=0.3,
)

# Define your query
query = "List the orders from the case AMALGAMATED vs MARSDEN"

flare_result = flare.run(query)

print(f"QUERY: {query}\n\n")
print(f"FLARE RESULT:\n    {flare_result}\n\n")

QUERY: List the orders from the case AMALGAMATED vs MARSDEN


FLARE RESULT:
     The orders from the case Amalgamated Television Services v Marsden Matter No 40005/97 [1998] NSWSC 4 (4 February 1998) were that the appeal was allowed in part. 




#### Use OpenAI LLM for comparison of not using FLARE

In [12]:
from langchain.llms import OpenAI

llm = OpenAI()
llm_result = llm(query)
print(f"LLM RESULT:\n    {llm_result}\n\n")

LLM RESULT:
    

1. Motion to Dismiss
2. Motion for Summary Judgment
3. Motion for Partial Summary Judgment
4. Motion to Compel Discovery
5. Motion to Strike 
6. Motion to Change Venue
7. Motion for Judgment on the Pleadings
8. Motion for Protective Order
9. Motion for Leave to File an Amended Complaint


